In [0]:
%pip install snowflake-connector-python
%pip install snowflake-snowpark-python
%pip install azure-storage-blob pandas


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, avg, split, year, month


Snowflake Connection

In [0]:
connection_parameters = {
  "account": "TPRTVOG-TG33465",
  "user": "shivashankari",
  "password": "Shivashankari_04",
  "warehouse": "RETAIL_WH",
  "database": "RETAIL_DB",
  "schema": "RETAIL_SCHEMA"
}
session = Session.builder.configs(connection_parameters).create()
print(" Connected to Snowflake")


 Connected to Snowflake


Connect to Azure Blob Storage and Download the Retail Sales CSV File

In [0]:
from azure.storage.blob import BlobServiceClient
import pandas as pd

blob_service_client = BlobServiceClient(account_url="https://hexadatastorage.blob.core.windows.net/", credential="sp=rwp&st=2025-10-22T16:16:28Z&se=2025-10-23T00:31:28Z&spr=https&sv=2024-11-04&sr=b&sig=UPuvW53PwcO8L6cGryIKNEaku8N6LiBgGUwOLAOlxYc%3D")
container_name = "retailsales"
blob_name = "Retail_Sales__500_rows__Preview.csv"

# Download file locally
with open("Retail_Sales.csv", "wb") as download_file:
    download_file.write(blob_service_client.get_blob_client(container=container_name, blob=blob_name).download_blob().readall())

print(" CSV downloaded from Azure")


 CSV downloaded from Azure


 Creating a Snowflake Stage and Read Data into a Snowpark DataFrame

In [0]:

session.sql("CREATE OR REPLACE STAGE AZURE_STAGE").collect()

# Put file to stage
session.file.put("Retail_Sales.csv", "@AZURE_STAGE", auto_compress=False)

# Read file from stage using Snowpark
df = session.read.options({
    "field_delimiter": ",",
    "skip_header": 1
}).csv("@AZURE_STAGE/Retail_Sales.csv")

df.show()


-------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"c1"          |"c2"        |"c3"     |"c4"      |"c5"           |"c6"   |"c7"     |"c8"     |"c9"             |"c10"        |"c11"  |"c12"  |"c13"    |"c14"     |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
|ORD-5F8D6F0C  |2024-10-08  |2024-10  |CUST1000  |Ananya Sharma  |India  |South    |Mumbai   |Office Supplies  |Paper        |9      |0.00   |2700.0   |780.43    |
|ORD-BF0078E4  |2024-08-11  |2024-08  |CUST1001  |Aarav Iyer     |India  |Central  |Lucknow  |Technology       |Networking   |4      |0.15   |27200.0  |4135.60   |
|ORD-86CD58A3  |2024-06-12  |2024-06  |CUST1002  |Arjun Sharma   |USA    |East     |Kolkata  |Furniture        |Tables       |4      |0.10   |31500.0  |5676.96   |
|ORD-FB0CD2D9  |

Load the DataFrame into Snowflake as a Raw Table (RETAIL_SALES_RAW)

In [0]:

df.write.mode("overwrite").save_as_table("RETAIL_SALES_RAW")

print("Data loaded to Snowflake table RETAIL_SALES_RAW")

     

Data loaded to Snowflake table RETAIL_SALES_RAW


**Transformations**

In [0]:
from snowflake.snowpark.functions import col

df_simple = df.with_column("DISCOUNTED_SALES", col('"c13"') * 0.9)
df_simple.show(5)



---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"c1"          |"c2"        |"c3"     |"c4"      |"c5"           |"c6"   |"c7"     |"c8"     |"c9"             |"c10"       |"c11"  |"c12"  |"c13"    |"c14"     |"DISCOUNTED_SALES"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|ORD-5F8D6F0C  |2024-10-08  |2024-10  |CUST1000  |Ananya Sharma  |India  |South    |Mumbai   |Office Supplies  |Paper       |9      |0.00   |2700.0   |780.43    |2430.00             |
|ORD-BF0078E4  |2024-08-11  |2024-08  |CUST1001  |Aarav Iyer     |India  |Central  |Lucknow  |Technology       |Networking  |4      |0.15   |27200.0  |4135.60   |24480.00            |
|ORD-86CD58A3  |2024-06-12  |2024-06  |CUST1002  |Arjun Sharma   |USA    |East  

In [0]:
df_model = (
    df.select(
        col('"c1"').alias("OrderID"),
        col('"c2"').cast("date").alias("OrderDate"),
        col('"c3"').alias("MonthOfSale"),
        col('"c4"').alias("CustomerID"),
        col('"c5"').alias("CustomerName"),
        col('"c6"').alias("Country"),
        col('"c7"').alias("Region"),
        col('"c8"').alias("City"),
        col('"c9"').alias("Category"),
        col('"c10"').alias("Subcategory"),
        col('"c11"').cast("NUMBER").alias("Quantity"),
        col('"c12"').cast("FLOAT").alias("Discount"),
        col('"c13"').cast("FLOAT").alias("Sales"),
        col('"c14"').cast("FLOAT").alias("Profit")
    )
)

df_model.write.mode("overwrite").save_as_table("RETAIL_SCHEMA.RETAIL_SALES_MODELED")
print("Modeled data saved to RETAIL_SCHEMA.RETAIL_SALES_MODELED")


Modeled data saved to RETAIL_SCHEMA.RETAIL_SALES_MODELED


In [0]:
from snowflake.snowpark.functions import when

df_transformed = df_model.with_column(
    "ProfitMargin",
    when(col("Sales") != 0, (col("Profit") / col("Sales")) * 100).otherwise(0)
)

df_transformed.write.mode("overwrite").save_as_table("RETAIL_SCHEMA.RETAIL_SALES_MODELED")
print("Added ProfitMargin column and updated table")

Added ProfitMargin column and updated table
